In [1]:
# ============================================================
# 012 Person Deep Dive
# ============================================================
# Purpose:
# This notebook is designed to support *initial meetings* with startup founders,
# executives, or key decision-makers by producing a concise, evidence-based
# briefing using lightweight public signals.
#
# Rather than attempting exhaustive web crawling or deep document parsing,
# this notebook prioritizes speed, robustness, and decision relevance.
#
# The core idea is:
#   "Good enough context, quickly, with clear sources and open questions."
#
# ------------------------------------------------------------
# What this notebook does
# ------------------------------------------------------------
# 1) Collects minimal user inputs (name, company, context)
# 2) Generates a small set of simple, high-signal search queries
# 3) Retrieves only top-ranked Google CSE results (title + snippet + URL)
# 4) Passes those results to an LLM for structured summarization and synthesis
# 5) Outputs a short meeting-ready brief with sources and validation questions
#
# ------------------------------------------------------------
# What this notebook intentionally does NOT do
# ------------------------------------------------------------
# - It does NOT attempt full biography reconstruction.
# - It does NOT scrape or deeply parse webpage HTML.
# - It does NOT infer private attributes or make definitive judgments.
# - It does NOT replace direct confirmation during the meeting.
#
# Any missing or ambiguous information is explicitly labeled as such
# and treated as a prompt for clarification, not speculation.
#
# ------------------------------------------------------------
# Intended use cases
# ------------------------------------------------------------
# - First meetings with founders or executives
# - Investment or partnership pre-reads
# - Rapid personal context building before live discussions
# - Hypothesis-driven conversation preparation
#
# This notebook is optimized for "speed to insight", not completeness.
#
# ------------------------------------------------------------
# Design principles
# ------------------------------------------------------------
# - Lightweight by default: prefer snippets over full documents
# - Source-first: every claim should point to a concrete URL
# - Hypothesis, not conclusion: outputs are meant to be tested in conversation
# - Transparency: search queries and sources are always visible
#
# ------------------------------------------------------------
# Output philosophy
# ------------------------------------------------------------
# The final output should help the user:
# - Avoid obvious questions already answered publicly
# - Ask sharper, more specific validation questions
# - Quickly understand how this individual presents themselves externally
# - Identify gaps or ambiguities worth exploring in the meeting
#
# In short, this notebook helps you walk into the room
# *informed, curious, and directionally prepared*—not overconfident.
# ============================================================


In [2]:
# ============================================================
# 1. Widget Input
# ============================================================
# Purpose:
# Capture minimal but high-signal inputs that guide search queries
# and downstream LLM synthesis.
#
# Inputs are intentionally lightweight.
# If a field is unknown, it can be left blank.
#
# The notebook is designed to work even with partial information.
# ============================================================


# ----------------------------
# 1-1. Imports
# ----------------------------
import ipywidgets as widgets
from dataclasses import dataclass
from IPython.display import display, Markdown


# ----------------------------
# 1-2. Input data model
# ----------------------------
@dataclass
class InputContext:
    person_name: str
    company_name: str
    meeting_context: str
    role_hint: str
    language_preference: str   # "en" or "ja"


# ----------------------------
# 1-3. Widgets
# ----------------------------

person_name_w = widgets.Text(
    description="Name",
    placeholder="e.g., Keisuke Nakatsuka",
    style={"description_width": "120px"},
    layout=widgets.Layout(width="450px"),
)

company_name_w = widgets.Text(
    description="Company",
    placeholder="e.g., B Capital",
    style={"description_width": "120px"},
    layout=widgets.Layout(width="450px"),
)

role_hint_w = widgets.Text(
    description="Role (optional)",
    placeholder="e.g., Founder, CEO, GM Japan",
    style={"description_width": "120px"},
    layout=widgets.Layout(width="450px"),
)

meeting_context_w = widgets.Textarea(
    description="Meeting context",
    placeholder=(
        "e.g., Initial investment discussion, partnership exploration, "
        "customer introduction, recruiting conversation"
    ),
    style={"description_width": "120px"},
    layout=widgets.Layout(width="650px", height="90px"),
)

language_w = widgets.Dropdown(
    description="Output language",
    options=[("English", "en"), ("Japanese", "ja")],
    value="en",
    style={"description_width": "120px"},
)

submit_button = widgets.Button(
    description="Confirm inputs",
    button_style="success",
    icon="check",
    layout=widgets.Layout(width="200px"),
)

input_out = widgets.Output()


# ----------------------------
# 1-4. State & handler
# ----------------------------
INPUT_STATE = {"context": None}

def _on_submit(_):
    with input_out:
        input_out.clear_output()

        INPUT_STATE["context"] = InputContext(
            person_name=person_name_w.value.strip(),
            company_name=company_name_w.value.strip(),
            role_hint=role_hint_w.value.strip(),
            meeting_context=meeting_context_w.value.strip(),
            language_preference=language_w.value,
        )

        ctx = INPUT_STATE["context"]

        # Basic validation (lightweight)
        if not ctx.person_name:
            display(Markdown("⚠️ **Name is required to proceed.**"))
            return

        display(Markdown("### ✅ Input confirmed"))
        display(Markdown(f"- **Name**: {ctx.person_name}"))
        display(Markdown(f"- **Company**: {ctx.company_name or '(not provided)'}"))
        if ctx.role_hint:
            display(Markdown(f"- **Role hint**: {ctx.role_hint}"))
        if ctx.meeting_context:
            display(Markdown(f"- **Meeting context**: {ctx.meeting_context}"))
        display(Markdown(f"- **Output language**: {ctx.language_preference}"))

submit_button.on_click(_on_submit)


# ----------------------------
# 1-5. Render UI
# ----------------------------
display(Markdown("### 1. Input"))
display(Markdown(
    "Provide minimal context to guide search and synthesis. "
    "Fields can be left blank if unknown."
))

display(widgets.VBox([
    person_name_w,
    company_name_w,
    role_hint_w,
    meeting_context_w,
    language_w,
    submit_button,
    input_out,
]))


# ----------------------------
# 1-6. Accessor
# ----------------------------
def get_input_context() -> InputContext:
    """
    Returns the confirmed input context.
    """
    ctx = INPUT_STATE.get("context")
    if ctx is None:
        raise ValueError("Input not confirmed. Please fill in the fields and click 'Confirm inputs'.")
    return ctx


### 1. Input

Provide minimal context to guide search and synthesis. Fields can be left blank if unknown.

In [4]:
# ============================================================
# 2. Query (Ultra-Simple)
# ============================================================
# Purpose:
# Generate a small set of high-signal search queries that are
# robust across languages and content types.
#
# This section intentionally avoids complex query logic.
# Fewer, clearer queries tend to outperform long, overfitted ones
# for initial discovery.
#
# Input:
# - InputContext from Section 1
#
# Output:
# - A short, ordered list of search queries to be sent to Google CSE
# ============================================================

# ----------------------------
# 2-0. Load API Keys
# ----------------------------
import os
from dotenv import load_dotenv

# Load env.txt explicitly
load_dotenv("env.txt")

# --- OpenAI ---
from openai import OpenAI

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if OPENAI_API_KEY is None:
    raise ValueError("OPENAI_API_KEY could not be loaded from env.txt.")
else:
    print("✅ OPENAI_API_KEY loaded successfully")

client = OpenAI(api_key=OPENAI_API_KEY)

# --- Google CSE ---
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
GOOGLE_CSE_CX = os.getenv("GOOGLE_CSE_CX")

if GOOGLE_API_KEY is None or GOOGLE_CSE_CX is None:
    raise ValueError("GOOGLE_API_KEY or GOOGLE_CSE_CX is missing in env.txt")
else:
    print("✅ Google CSE credentials loaded successfully")

# ----------------------------
# 2-1. Load input context
# ----------------------------
ctx = get_input_context()

display(Markdown("### 2. Query Generation"))
display(Markdown("Generating a minimal set of high-signal search queries."))


# ----------------------------
# 2-2. Query builder
# ----------------------------
def build_simple_queries(ctx) -> list[str]:
    """
    Build a small number of clear, intent-driven search queries.
    Queries are language-aware but remain simple.
    """
    name = ctx.person_name.strip()
    company = ctx.company_name.strip()
    role = ctx.role_hint.strip()

    base = f'"{name}"'
    company_clause = f' "{company}"' if company else ""
    role_clause = f' "{role}"' if role else ""

    queries = []

    # --- Biography / background ---
    queries.append(f'{base}{company_clause} 略歴')
    queries.append(f'{base}{company_clause} 経歴')

    # --- Interviews / thought leadership ---
    queries.append(f'{base}{company_clause} interview')
    queries.append(f'{base}{company_clause} インタビュー')

    # --- Talks / public appearances ---
    queries.append(f'{base}{company_clause} speaker')
    queries.append(f'{base}{company_clause} 登壇')

    # --- LinkedIn anchor (identity check) ---
    queries.append(f'{base}{company_clause} LinkedIn')

    # Optional role disambiguation
    if role:
        queries.append(f'{base}{role_clause}{company_clause}')

    # De-duplicate while preserving order
    seen = set()
    clean_queries = []
    for q in queries:
        q2 = " ".join(q.split())
        if q2 not in seen:
            clean_queries.append(q2)
            seen.add(q2)

    return clean_queries


queries = build_simple_queries(ctx)


# ----------------------------
# 2-3. Review queries (transparency)
# ----------------------------
display(Markdown("#### Generated queries"))
for i, q in enumerate(queries, start=1):
    display(Markdown(f"{i}. `{q}`"))


# ----------------------------
# 2-4. Accessor
# ----------------------------
def get_search_queries() -> list[str]:
    """
    Returns the generated search queries.
    """
    return queries


✅ OPENAI_API_KEY loaded successfully
✅ Google CSE credentials loaded successfully


### 2. Query Generation

Generating a minimal set of high-signal search queries.

#### Generated queries

1. `"Pham Quang Cuong" "Eureka Robotics" 略歴`

...

4. `"Pham Quang Cuong" "Eureka Robotics" インタビュー`

5. `"Pham Quang Cuong" "Eureka Robotics" speaker`

6. `"Pham Quang Cuong" "Eureka Robotics" 登壇`

7. `"Pham Quang Cuong" "Eureka Robotics" LinkedIn`

8. `"Pham Quang Cuong" "CEO" "Eureka Robotics"`

In [5]:
# ============================================================
# 3. Retrieve Top Results via Google CSE (Multi-site discovery, no LinkedIn deep-dive)
# ============================================================
# Purpose:
# Run a small set of base queries via Google CSE and collect top results (title/snippet/url).
# Then run a second wave of *multi-site* discovery queries to broaden coverage across:
# - news / press releases
# - conferences / events / webinars
# - podcasts / video (YouTube)
# - blogs / note / Medium
# - company pages / partner pages
#
# Key idea:
# - LinkedIn may appear in results, but we do NOT "deep dive" LinkedIn via site-fixed queries.
# - We optimize for diversity and breadth of sources, while keeping payload lightweight.
#
# Output:
# - search_results_df: deduped table of results with metadata
# - Human-readable preview (Title / Snippet / URL)
# ============================================================

import time
import pandas as pd
from urllib.parse import urlparse
from IPython.display import display, Markdown


# ----------------------------
# 3-1. Load inputs
# ----------------------------
ctx = get_input_context()
base_queries = get_search_queries()

display(Markdown("### 3. Google CSE Retrieval"))
display(Markdown(f"Running **{len(base_queries)}** base queries and collecting top results."))


# ----------------------------
# 3-2. CSE request helper
# ----------------------------
def cse_search(query: str, api_key: str, cx: str, num: int = 5, start: int = 1) -> list[dict]:
    """
    Google Programmable Search Engine (CSE) call.
    num: 1..10
    start: 1, 1+num, 1+2*num, ... (1-based index)
    """
    import requests

    url = "https://www.googleapis.com/customsearch/v1"
    params = {
        "key": api_key,
        "cx": cx,
        "q": query,
        "num": max(1, min(num, 10)),
        "start": start,
    }
    r = requests.get(url, params=params, timeout=30)
    r.raise_for_status()
    return r.json().get("items", [])


def get_domain(url: str) -> str:
    try:
        return (urlparse(url).netloc or "").lower().replace("www.", "")
    except Exception:
        return ""


# ----------------------------
# 3-3. Filtering rules (lightweight)
# ----------------------------
def is_low_signal_url(url: str) -> bool:
    """
    Filter out pages that are usually not useful for bio/interview discovery.
    Adjust as needed.
    """
    u = (url or "").lower()

    # LinkedIn directory pages (high noise)
    if "linkedin.com/pub/dir" in u or "/pub/dir" in u:
        return True

    # Generic search pages
    if "google.com/search" in u:
        return True

    return False


# ----------------------------
# 3-4. Retrieval parameters
# ----------------------------
TOP_K_PER_PAGE = 5
MAX_PAGES_PER_QUERY = 2
MAX_TOTAL_RESULTS = 80
MAX_PER_DOMAIN = 5
SLEEP_SEC = 0.25

rows = []
seen_urls = set()
domain_counter = {}


def can_add_domain(domain: str) -> bool:
    return domain_counter.get(domain, 0) < MAX_PER_DOMAIN


def add_items_to_rows(items: list[dict], qi: int, q: str, start_rank: int, wave: str):
    """
    Add CSE items to rows with dedup + domain cap.
    """
    for rank, it in enumerate(items, start=start_rank):
        url = (it.get("link") or "").strip()
        if not url or url in seen_urls:
            continue
        if is_low_signal_url(url):
            continue

        domain = get_domain(url)
        if not can_add_domain(domain):
            continue

        seen_urls.add(url)
        domain_counter[domain] = domain_counter.get(domain, 0) + 1

        rows.append({
            "wave": wave,  # "base" or "second"
            "query_idx": qi,
            "query": q,
            "rank_in_query": rank,
            "title": (it.get("title") or "")[:250],
            "snippet": (it.get("snippet") or "")[:900],
            "url": url,
            "domain": domain,
        })

        if len(rows) >= MAX_TOTAL_RESULTS:
            break


# ----------------------------
# 3-5. Base wave
# ----------------------------
for qi, q in enumerate(base_queries, start=1):
    for page in range(MAX_PAGES_PER_QUERY):
        start = 1 + page * TOP_K_PER_PAGE
        try:
            items = cse_search(
                q,
                GOOGLE_API_KEY,
                GOOGLE_CSE_CX,
                num=TOP_K_PER_PAGE,
                start=start,
            )
            print(f"[DEBUG] base q='{q}' page={page} start={start} items={len(items)}")
        except Exception as e:
            print(f"[CSE ERROR] base q='{q}' start={start} error={e}")
            continue

        add_items_to_rows(items, qi=qi, q=q, start_rank=start, wave="base")
        time.sleep(SLEEP_SEC)

        if len(rows) >= MAX_TOTAL_RESULTS:
            break
    if len(rows) >= MAX_TOTAL_RESULTS:
        break


# ----------------------------
# 3-6. Second wave: multi-site discovery
# ----------------------------
def build_multisite_second_wave_queries(ctx) -> list[str]:
    """
    Broaden discovery across multiple content types and platforms.
    Keep queries compact and intent-driven.
    """
    name = ctx.person_name.strip()
    company = (ctx.company_name or "").strip()
    role = (ctx.role_hint or "").strip()

    base = f'"{name}"'
    company_clause = f' "{company}"' if company else ""
    role_clause = f' "{role}"' if role else ""

    # Intent keywords: interviews / talks / press / profiles
    intents = [
        "interview", "インタビュー",
        "speaker", "登壇", "講演", "webinar", "セミナー",
        "profile", "経歴", "略歴",
        "podcast", "YouTube",
    ]

    # Site targets: add selectively (not mandatory, but helps diversify)
    # Use domain-level site filters (not path-level).
    sites = [
        "site:prtimes.jp",
        "site:note.com",
        "site:medium.com",
        "site:youtube.com",
        "site:podcasts.apple.com",
        "site:spotify.com",
        "site:nikkan.co.jp",
    ]

    queries = []

    # Generic high-signal queries (no site constraint)
    queries += [
        f"{base}{company_clause}{role_clause} interview",
        f"{base}{company_clause}{role_clause} インタビュー",
        f"{base}{company_clause} 登壇",
        f"{base}{company_clause} speaker",
        f"{base}{company_clause} 経歴",
        f"{base} 経歴",
    ]

    # Site-targeted diversification (lightweight)
    for s in sites:
        queries.append(f"{s} {base}{company_clause} インタビュー")
        queries.append(f"{s} {base}{company_clause} 登壇")
        queries.append(f"{s} {base}{company_clause} profile")

    # De-duplicate preserving order
    seen = set()
    out = []
    for q in queries:
        q2 = " ".join(q.split())
        if q2 not in seen:
            out.append(q2)
            seen.add(q2)

    return out


second_queries = build_multisite_second_wave_queries(ctx)

display(Markdown("### 3.2 Second-wave retrieval (multi-site discovery)"))
display(Markdown(f"Running **{len(second_queries)}** second-wave queries to broaden coverage."))

qi_offset = len(base_queries)

# Keep second wave relatively light
SECOND_WAVE_TOP_K = 5
SECOND_WAVE_PAGES = 1

for j, q2 in enumerate(second_queries, start=1):
    qi2 = qi_offset + j

    for page in range(SECOND_WAVE_PAGES):
        start = 1 + page * SECOND_WAVE_TOP_K
        try:
            items = cse_search(
                q2,
                GOOGLE_API_KEY,
                GOOGLE_CSE_CX,
                num=SECOND_WAVE_TOP_K,
                start=start,
            )
            print(f"[DEBUG] 2nd q='{q2}' start={start} items={len(items)}")
        except Exception as e:
            print(f"[CSE ERROR] 2nd q='{q2}' start={start} error={e}")
            continue

        add_items_to_rows(items, qi=qi2, q=q2, start_rank=start, wave="second")
        time.sleep(SLEEP_SEC)

        if len(rows) >= MAX_TOTAL_RESULTS:
            break

    if len(rows) >= MAX_TOTAL_RESULTS:
        break


search_results_df = pd.DataFrame(rows)


# ----------------------------
# 3-7. Summary + preview
# ----------------------------
display(Markdown(
    f"#### Collected results: **{len(search_results_df)}** "
    f"(deduped, max {MAX_PER_DOMAIN} per domain)"
))

if not search_results_df.empty:
    display(search_results_df[["wave", "query_idx", "rank_in_query", "domain", "title", "url"]].head(30))
else:
    display(Markdown("⚠️ No results collected. Check CSE credentials and queries."))


def render_search_results_preview(df: pd.DataFrame, max_items: int = 25):
    display(Markdown("### 🔍 Search Results Preview (Title / Snippet / URL)"))
    if df.empty:
        display(Markdown("⚠️ No results to display."))
        return

    for k, (_, row) in enumerate(df.head(max_items).iterrows(), start=1):
        title = row.get("title") or "(no title)"
        snippet = row.get("snippet") or "(no snippet)"
        url = row.get("url") or ""
        display(Markdown(f"""
**{k}. {title}**

{snippet}

🔗 {url}

---
"""))

render_search_results_preview(search_results_df, max_items=25)


# ----------------------------
# 3-8. Accessor
# ----------------------------
def get_search_results_df() -> pd.DataFrame:
    """
    Returns the CSE top results as a DataFrame.
    """
    return search_results_df


### 3. Google CSE Retrieval

Running **8** base queries and collecting top results.

[DEBUG] base q='"Pham Quang Cuong" "Eureka Robotics" 略歴' page=0 start=1 items=5
[DEBUG] base q='"Pham Quang Cuong" "Eureka Robotics" 略歴' page=1 start=6 items=1
[DEBUG] base q='"Pham Quang Cuong" "Eureka Robotics" 経歴' page=0 start=1 items=5
[DEBUG] base q='"Pham Quang Cuong" "Eureka Robotics" 経歴' page=1 start=6 items=5
[DEBUG] base q='"Pham Quang Cuong" "Eureka Robotics" interview' page=0 start=1 items=5
[DEBUG] base q='"Pham Quang Cuong" "Eureka Robotics" interview' page=1 start=6 items=5
[DEBUG] base q='"Pham Quang Cuong" "Eureka Robotics" インタビュー' page=0 start=1 items=5
[DEBUG] base q='"Pham Quang Cuong" "Eureka Robotics" インタビュー' page=1 start=6 items=0
[DEBUG] base q='"Pham Quang Cuong" "Eureka Robotics" speaker' page=0 start=1 items=5
[DEBUG] base q='"Pham Quang Cuong" "Eureka Robotics" speaker' page=1 start=6 items=5
[DEBUG] base q='"Pham Quang Cuong" "Eureka Robotics" 登壇' page=0 start=1 items=4
[DEBUG] base q='"Pham Quang Cuong" "Eureka Robotics" 登壇' page=1 start=6 items=0
[DEBUG] 

### 3.2 Second-wave retrieval (multi-site discovery)

Running **27** second-wave queries to broaden coverage.

...


**21. Eureka Robotics' Archimedes | Interview with Pham Quang Cuong ...**

Oct 20, 2019 ... Asian Scientist Magazine interviews Pham Quang Cuong, who shares about Eureka Robotics' newly launched robot, Archimedes, which possesses a ...

🔗 https://www.youtube.com/watch?v=yM1L3dFbfF8

---



**22. Eureka Robotics on Instagram: " Eureka Tokyo is growing fast ...**

Jun 27, 2025 ... ... interview with our CEO Pham Quang Cuong a few weeks ago. The ... CEOの Pham Quang Cuongが、まずEureka Roboticsの成り立ちや技術的独自 ...

🔗 https://www.instagram.com/p/DLZXxs0SjQ2/

---



**23. Eureka Robotics | Singapore Singapore**

... Eureka Robotics following an interview with our CEO Pham Quang Cuong a few weeks ago. The article discusses one of the industry's outstanding challenges ...

🔗 https://www.facebook.com/eurekarobotics/

---



**24. Eureka Robotics Pte. Ltd.｜Our Companies｜UTEC-The University ...**

Eureka Robotics Pte. Ltd.｜Our Companies of Introduction.UTEC is a seed ... Pham Quang Cuong is an Associate Professor of Robotics at NTU (on leave), a ...

🔗 https://www.ut-ec.co.jp/english/our_companies/eureka-robotics/

---



**25. Physical AIは現場で完成する ― Eureka Roboticsが選ぶ道｜株式 ...**

4 hours ago ... ... Pham Quang Cuong ）さん（以下、クオンさん）を迎え、TORCH代表の山内 ... Eureka Roboticsにとって、日本市場への進出状況はいかがでしょうか？

🔗 https://note.com/torch_inc/n/n2ab26d800197

---


In [6]:
# ============================================================
# 3.5 Fetch & Parse (Lightweight Crawler for Crawl-friendly Sites)
# ============================================================
# Purpose:
# Convert a subset of CSE results (URL list) into richer evidence by fetching
# the page HTML and extracting main text (no heavy crawling).
#
# Philosophy:
# - Only fetch a small number of URLs (top-N) to control time/cost.
# - Avoid sites that are usually login-walled / anti-bot (LinkedIn, X, Facebook).
# - Be polite: robots.txt check + rate limiting + clear User-Agent.
#
# Input:
# - search_results_df from Section 3
#
# Output:
# - pages_df: table with fetched text per URL (url, domain, title, extracted_text, status, etc.)
# - pages_payload: compact list for LLM (url, title, snippet, extracted_text excerpt)
# ============================================================

import re
import time
import requests
import pandas as pd
from urllib.parse import urlparse, urljoin
from IPython.display import display, Markdown


# ----------------------------
# 3.5-1. Config
# ----------------------------
FETCH_TOP_N = 15              # how many URLs to fetch (top rows of search_results_df)
MAX_TEXT_CHARS = 6000         # cap extracted text per page (for LLM payload)
TIMEOUT_SEC = 20
SLEEP_BETWEEN_REQ_SEC = 0.8   # be polite
USER_AGENT = "researchOSv2/1.0 (contact: you@example.com)"  # customize if you want


# Blocklist: high-friction / login-walled / unstable for text extraction
BLOCKLIST_DOMAINS = {
    "linkedin.com", "jp.linkedin.com",
    "facebook.com", "m.facebook.com",
    "x.com", "twitter.com",
    "instagram.com",
}

# Optionally: if you want strict allowlist mode, set ALLOWLIST_DOMAINS to a non-empty set.
# Example:
# ALLOWLIST_DOMAINS = {"prtimes.jp", "yourcompany.com", "nikkan.co.jp"}
ALLOWLIST_DOMAINS = set()


# ----------------------------
# 3.5-2. Helpers
# ----------------------------
def decode_html_response(r: requests.Response) -> tuple[str, str]:
    """
    Decode HTML response bytes into text with best-effort encoding detection.
    Returns: (decoded_text, encoding_used)
    """
    raw = r.content or b""
    ct = (r.headers.get("Content-Type") or "").lower()

    # 1) Content-Type charset
    enc = None
    m = re.search(r"charset=([a-z0-9\-\_]+)", ct)
    if m:
        enc = m.group(1).strip()

    # 2) requests guess
    if not enc:
        try:
            enc = r.apparent_encoding  # uses chardet/charset-normalizer underneath
        except Exception:
            enc = None

    # 3) Try decode with candidates
    candidates = []
    if enc:
        candidates.append(enc)
    candidates += ["utf-8", "cp932", "shift_jis", "euc-jp"]

    last_err = None
    for e in candidates:
        try:
            text = raw.decode(e, errors="replace")
            return text, e
        except Exception as ex:
            last_err = ex
            continue

    # Final fallback (should rarely happen)
    return raw.decode("utf-8", errors="replace"), "utf-8(fallback)"


def normalize_domain(d: str) -> str:
    return (d or "").lower().replace("www.", "").strip()


def domain_of(url: str) -> str:
    try:
        return normalize_domain(urlparse(url).netloc)
    except Exception:
        return ""


def should_fetch_url(url: str) -> tuple[bool, str]:
    """
    Decide whether to fetch a URL.
    Returns: (bool, reason)
    """
    dom = domain_of(url)

    if not url or not dom:
        return False, "missing_url_or_domain"

    if dom in BLOCKLIST_DOMAINS:
        return False, "blocked_domain"

    if ALLOWLIST_DOMAINS and dom not in ALLOWLIST_DOMAINS:
        return False, "not_in_allowlist"

    # Skip obvious non-HTML endpoints (optional)
    lower = url.lower()
    if any(lower.endswith(ext) for ext in [".pdf", ".jpg", ".jpeg", ".png", ".gif", ".zip"]):
        return False, "non_html_asset"

    return True, "ok"


def robots_allows(url: str, user_agent: str = USER_AGENT) -> tuple[bool, str]:
    """
    Lightweight robots.txt check using urllib.robotparser.
    If robots cannot be fetched/parsed, we default to allow (soft-fail) but record reason.
    """
    import urllib.robotparser as robotparser

    try:
        p = urlparse(url)
        robots_url = urljoin(f"{p.scheme}://{p.netloc}", "/robots.txt")
        rp = robotparser.RobotFileParser()
        rp.set_url(robots_url)
        rp.read()
        allowed = rp.can_fetch(user_agent, url)
        return allowed, "robots_ok" if allowed else "robots_disallow"
    except Exception as e:
        return True, f"robots_check_failed:{type(e).__name__}"


def extract_visible_text_simple(html: str) -> str:
    """
    Extract readable-ish text from HTML without heavy dependencies.
    Strategy:
      - remove script/style/noscript/svg
      - prefer <article> or <main> if available
      - else collect <p>, <li>, <h1-3> text
    """
    try:
        from bs4 import BeautifulSoup
    except ImportError:
        # Fallback: strip tags crudely (less ideal)
        text = re.sub(r"<[^>]+>", " ", html)
        text = re.sub(r"\s+", " ", text).strip()
        return text

    soup = BeautifulSoup(html, "html.parser")

    # drop noisy nodes
    for tag in soup(["script", "style", "noscript", "svg", "header", "footer", "nav", "form"]):
        tag.decompose()

    # prefer main/article
    main = soup.find("article") or soup.find("main")
    scope = main if main else soup.body if soup.body else soup

    parts = []
    for el in scope.find_all(["h1", "h2", "h3", "p", "li"]):
        t = el.get_text(" ", strip=True)
        if t and len(t) >= 20:
            parts.append(t)

    text = "\n".join(parts)
    text = re.sub(r"\n{3,}", "\n\n", text).strip()
    return text


def truncate_text(text: str, max_chars: int) -> str:
    t = (text or "").strip()
    if len(t) <= max_chars:
        return t
    return t[:max_chars].rstrip() + "\n...[truncated]"


def fetch_one(url: str) -> dict:
    """
    Fetch a single URL and extract text.
    Returns a row dict with status and extracted text.
    """
    ok, reason = should_fetch_url(url)
    if not ok:
        return {
            "url": url,
            "domain": domain_of(url),
            "fetch_ok": False,
            "fetch_reason": reason,
            "http_status": None,
            "content_type": None,
            "encoding": "",
            "title": "",
            "extracted_text": "",
        }

    allow, robots_reason = robots_allows(url)
    if not allow:
        return {
            "url": url,
            "domain": domain_of(url),
            "fetch_ok": False,
            "fetch_reason": robots_reason,
            "http_status": None,
            "content_type": None,
            "encoding": "",
            "title": "",
            "extracted_text": "",
        }

    headers = {"User-Agent": USER_AGENT, "Accept": "text/html,application/xhtml+xml"}
    try:
        r = requests.get(url, headers=headers, timeout=TIMEOUT_SEC)
        ct = (r.headers.get("Content-Type") or "").lower()
        status = r.status_code

        if status >= 400:
            return {
                "url": url,
                "domain": domain_of(url),
                "fetch_ok": False,
                "fetch_reason": f"http_{status}",
                "http_status": status,
                "content_type": ct,
                "encoding": "",
                "title": "",
                "extracted_text": "",
            }

        # Only parse HTML-ish
        if "text/html" not in ct and "application/xhtml+xml" not in ct and ct:
            return {
                "url": url,
                "domain": domain_of(url),
                "fetch_ok": False,
                "fetch_reason": "non_html_content_type",
                "http_status": status,
                "content_type": ct,
                "encoding": "",
                "title": "",
                "extracted_text": "",
            }

        html_text, enc_used = decode_html_response(r)

        extracted = extract_visible_text_simple(html_text)
        extracted = truncate_text(extracted, MAX_TEXT_CHARS)

        # title (best-effort)
        title = ""
        try:
            from bs4 import BeautifulSoup
            soup = BeautifulSoup(html_text, "html.parser")
            title = (soup.title.get_text(strip=True) if soup.title else "")[:200]
        except Exception:
            title = ""

        return {
            "url": url,
            "domain": domain_of(url),
            "fetch_ok": True,
            "fetch_reason": "ok",
            "http_status": status,
            "content_type": ct,
            "encoding": enc_used,
            "title": title,
            "extracted_text": extracted,
        }

    except Exception as e:
        return {
            "url": url,
            "domain": domain_of(url),
            "fetch_ok": False,
            "fetch_reason": f"exception:{type(e).__name__}",
            "http_status": None,
            "content_type": None,
            "encoding": "",
            "title": "",
            "extracted_text": "",
        }



# ----------------------------
# 3.5-3. Run fetch on top-N CSE results
# ----------------------------
search_results_df = get_search_results_df()

display(Markdown("### 3.5 Fetch & Parse"))
if search_results_df.empty:
    raise ValueError("search_results_df is empty. Run Section 3 first.")

# Select top-N URLs (you can change ordering if you want)
to_fetch_df = search_results_df.dropna(subset=["url"]).head(FETCH_TOP_N).copy()

display(Markdown(f"Fetching **{len(to_fetch_df)}** URLs (top-{FETCH_TOP_N})."))
display(to_fetch_df[["query_idx", "rank_in_query", "domain", "title", "url"]].head(10))

fetched_rows = []
for i, row in to_fetch_df.iterrows():
    url = row["url"]
    out = fetch_one(url)

    # Carry snippet/title from CSE for context
    out["cse_title"] = row.get("title", "")
    out["cse_snippet"] = row.get("snippet", "")
    out["query_idx"] = row.get("query_idx", None)
    out["rank_in_query"] = row.get("rank_in_query", None)

    fetched_rows.append(out)
    time.sleep(SLEEP_BETWEEN_REQ_SEC)

pages_df = pd.DataFrame(fetched_rows)

display(Markdown(
    f"#### Fetched pages: **{pages_df['fetch_ok'].sum()}** success / **{len(pages_df)}** total"
))
display(pages_df[["fetch_ok", "fetch_reason", "domain", "encoding", "title", "url"]].head(20))



# ----------------------------
# 3.5-4. Build payload for LLM (snippet + extracted text)
# ----------------------------
def build_pages_payload(pages_df: pd.DataFrame, max_pages: int = 10, max_text_chars: int = 3000) -> list[dict]:
    """
    Build a compact payload for the LLM using:
    - url
    - CSE title/snippet
    - extracted_text (truncated)
    """
    df = pages_df.copy()
    df = df[df["fetch_ok"] == True].copy()

    # Optional: prioritize longer extracted text (more content)
    df["text_len"] = df["extracted_text"].fillna("").map(len)
    df = df.sort_values(["text_len"], ascending=False).head(max_pages)

    payload = []
    for _, r in df.iterrows():
        payload.append({
            "url": r["url"],
            "domain": r["domain"],
            "title": (r["title"] or r.get("cse_title") or "")[:250],
            "snippet": (r.get("cse_snippet") or "")[:800],
            "extracted_text": truncate_text(r.get("extracted_text") or "", max_text_chars),
        })
    return payload


pages_payload = build_pages_payload(pages_df, max_pages=10, max_text_chars=3000)

display(Markdown(f"#### Pages payload for LLM: **{len(pages_payload)}** pages"))
if pages_payload:
    display(Markdown("Showing the first payload item (truncated):"))
    display(pages_payload[0])


# ----------------------------
# 3.5-5. Accessors
# ----------------------------
def get_pages_df() -> pd.DataFrame:
    """
    Returns fetched pages and extracted text.
    """
    return pages_df

def get_pages_payload() -> list[dict]:
    """
    Returns compact payload to enrich LLM input (snippet + extracted text).
    """
    return pages_payload


### 3.5 Fetch & Parse

Fetching **15** URLs (top-15).

,query_idx,rank_in_query,domain,title,url
0,1,1,creators.spotify.com,#1 日本最強エンジニア集団ALGO ARTIS。極限まで自由に考える知 ...,https://creators.spotify.com/pod/profile/utec/...
1,1,2,jp.linkedin.com,Eureka Robotics | LinkedIn,https://jp.linkedin.com/company/eurekarobotics
2,1,3,scholar.google.es,‪Jingyi Yang‬ - ‪Google Scholar‬,https://scholar.google.es/citations?user=AEINg...
3,1,4,scholar.google.com,Shohei Fujii - Google Scholar,https://scholar.google.com/citations?user=fh3r...
4,1,5,instagram.com,NTU College of Engineering (@ntucoe) · Singapo...,https://www.instagram.com/ntucoe/
5,1,6,ut-ec.co.jp,Untitled,https://www.ut-ec.co.jp/admin/wp-content/uploa...
6,2,1,sg.linkedin.com,Eureka Robotics | LinkedIn,https://sg.linkedin.com/company/eurekarobotics
7,2,2,ut-ec.co.jp,Eureka Robotics Pte. Ltd.｜投資先企業｜UTEC-東京大学エッジ ...,https://www.ut-ec.co.jp/our_companies/eureka-r...
8,2,3,startupclass.co.jp,Eureka Robotics株式会社の転職・求人情報 | スタクラ,https://startupclass.co.jp/online/companies/1742/
9,2,4,prtimes.jp,"Eureka Robotics、シリーズＡラウンドで1,050万ドルの資金調達を ...",https://prtimes.jp/main/html/rd/p/000000001.00...


#### Fetched pages: **7** success / **15** total

,fetch_ok,fetch_reason,domain,encoding,title,url
0,True,ok,creators.spotify.com,utf-8,#1 日本最強エンジニア集団ALGO ARTIS。極限まで自由に考える知の実験【ゲスト：AL...,https://creators.spotify.com/pod/profile/utec/...
1,False,blocked_domain,jp.linkedin.com,,,https://jp.linkedin.com/company/eurekarobotics
2,False,robots_disallow,scholar.google.es,,,https://scholar.google.es/citations?user=AEINg...
3,False,robots_disallow,scholar.google.com,,,https://scholar.google.com/citations?user=fh3r...
4,False,blocked_domain,instagram.com,,,https://www.instagram.com/ntucoe/
5,False,non_html_asset,ut-ec.co.jp,,,https://www.ut-ec.co.jp/admin/wp-content/uploa...
6,False,robots_disallow,sg.linkedin.com,,,https://sg.linkedin.com/company/eurekarobotics
7,True,ok,ut-ec.co.jp,utf-8,Eureka Robotics Pte. Ltd.｜投資先企業｜UTEC-東京大学エッジキャ...,https://www.ut-ec.co.jp/our_companies/eureka-r...
8,True,ok,startupclass.co.jp,utf-8,Eureka Robotics株式会社の転職・求人情報 | スタクラ,https://startupclass.co.jp/online/companies/1742/
9,True,ok,prtimes.jp,utf-8,"Eureka Robotics、シリーズＡラウンドで1,050万ドルの資金調達を実施 | エ...",https://prtimes.jp/main/html/rd/p/000000001.00...


#### Pages payload for LLM: **7** pages

Showing the first payload item (truncated):

{'url': 'https://creators.spotify.com/pod/profile/utec/episodes/2-ALGO-ARTISALGO-ARTIS-e2l0qs7',
 'domain': 'creators.spotify.com',
 'title': '#1 日本最強エンジニア集団ALGO ARTIS。極限まで自由に考える知の実験【ゲスト：ALGO ARTIS 永田健太郎氏】 by Tech Startupの舞台裏',
 'snippet': 'ゲスト経歴中沢冬芽1998年生まれ。長野県松本市出身。 東京大学法学部在学中に ... We are thrilled to have Dr. Pham Quang Cuong, CEO of Eureka Robotics, the\xa0...',
 'extracted_text': '#1 日本最強エンジニア集団ALGO ARTIS。極限まで自由に考える知の実験【ゲスト：ALGO ARTIS 永田健太郎氏】\n東証グロース市場の新しい上場維持基準「5年で時価総額100億円」を起点に、IPOの前提がどう変わるのかをUTEC内でざっくばらんに整理。短期の痛みと中長期の意義、IPO準備の“入り口の高さ”の変化、M&A・セカンダリーを含む出口の多様化、そしてVCに求められるケイパビリティの変化まで、実務目線で語ってみます。\n「5年で100億」って、要は何が変わる？\n「アメリカと比べすぎ問題」をどう考えるか\n“いいIPO”って、立場によって見え方が違うよね、という話\n上場後に伸びている会社に共通していること\nVC自身もやり方をアップデートする必要ある？\nスピーダ https://initial.inc/articles/growth-market-post-ipo\nJETRO https://www.jetro.go.jp/world/reports/2025/02/8d88f1c5f60ab2b1.html\n★UTECへの投資・起業のご相談はこちら\nhttps://share.hsforms.com/1_WfJXAagSWmKnsxVxSalXQc3fla\n★UTEC投資先参画や研究機関発スタートアップインキュベーション支援に関心のある方向けのコミュニティSOC（Start

In [7]:
# ============================================================
# 4. Pass Results to LLM (pages_payload-first)
# ============================================================

import os
import json
from IPython.display import display, Markdown
from openai import OpenAI

client = OpenAI(api_key=OPENAI_API_KEY)


# ----------------------------
# 4-2. Load inputs
# ----------------------------
ctx = get_input_context()
search_results_df = get_search_results_df()

display(Markdown("### 4. LLM Synthesis"))

if pages_payload and len(pages_payload) > 0:
    display(Markdown(f"Using **{len(pages_payload)}** fetched pages (snippet + extracted text)."))
else:
    display(Markdown(
        f"No fetched-page payload found. Falling back to **{len(search_results_df)}** search results (title/snippet only)."
    ))

if (not pages_payload or len(pages_payload) == 0) and search_results_df.empty:
    raise ValueError("No evidence available. Run Section 3 (and optionally 3.5) first.")


# ----------------------------
# 4-3. Prepare payload for LLM
# ----------------------------
def build_llm_payload_from_search_results(df, max_items: int = 25):
    """
    Fallback payload: title/snippet/url only.
    """
    out = []
    if df is None or df.empty:
        return out

    for _, row in df.head(max_items).iterrows():
        out.append({
            "url": row.get("url", ""),
            "domain": row.get("domain", ""),
            "title": row.get("title", ""),
            "snippet": row.get("snippet", ""),
            "query_idx": int(row.get("query_idx", 0)) if str(row.get("query_idx", "")).isdigit() else None,
            "rank_in_query": int(row.get("rank_in_query", 0)) if str(row.get("rank_in_query", "")).isdigit() else None,
            "extracted_text": "",  # none in fallback
        })
    return out


# Preferred: pages_payload already has extracted_text
if pages_payload and len(pages_payload) > 0:
    llm_payload = pages_payload
else:
    llm_payload = build_llm_payload_from_search_results(search_results_df, max_items=25)


# ----------------------------
# 4-4. Prompt construction
# ----------------------------
def normalize_lang(lang: str) -> str:
    l = (lang or "").strip().lower()
    if l in ["ja", "jp", "japanese", "日本語"]:
        return "ja"
    if l in ["en", "english", "英語"]:
        return "en"
    return "ja"  # default


def build_llm_prompt(ctx, llm_payload):
    lang = normalize_lang(getattr(ctx, "language_preference", "ja"))

    output_language_instruction = (
        "Write ALL human-readable text values in Japanese."
        if lang == "ja"
        else "Write ALL human-readable text values in English."
    )

    # Also enforce that JSON keys remain in English (structure fixed)
    structure_instruction = (
        "Keep the JSON keys exactly as specified (English keys). "
        "Only the string VALUES should follow the output language rule."
    )

    return f"""
You are assisting with preparation for an initial business meeting.

Target person:
- Name: {ctx.person_name}
- Company: {ctx.company_name}
- Role hint (if any): {ctx.role_hint or "(not provided)"}

Meeting context:
{ctx.meeting_context or "(not provided)"}

Output language rule:
- {output_language_instruction}
- {structure_instruction}

Task:
Using ONLY the evidence records provided below (title/snippet/url and, when available, extracted_text),
produce a structured, meeting-ready briefing.

Important assumptions:
- Evidence may include items that are weakly related or not related to the target person.
- You are NOT required to use all evidence.
- Prefer precision over coverage.
- It is acceptable to ignore evidence that is not clearly about the target person.

Rules:
- Do NOT invent facts.
- If information is not clearly supported by evidence, set it to "not found".
- Every factual statement must reference one or more URLs (put URLs in "sources"/"evidence").
- Hypotheses must be labeled as hypotheses, not conclusions.
- Avoid generic statements that could apply to anyone.
- If evidence is thin or ambiguous, state that explicitly.

Return ONLY valid JSON matching this schema exactly:

{{
  "profile_summary": {{
    "current_role": "... or 'not found'",
    "background_overview": "... (brief, factual)"
  }},
  "career_highlights": [
    {{
      "point": "...",
      "sources": ["url1", "url2"]
    }}
  ],
  "public_themes": [
    {{
      "theme": "...",
      "evidence": ["url1"]
    }}
  ],
  "hypotheses": [
    {{
      "hypothesis": "...",
      "rationale": "...",
      "sources": ["url1"],
      "confidence": 0.0
    }}
  ],
  "talking_points": [
    {{
      "question": "...",
      "intent": "...",
      "related_hypothesis": "..."
    }}
  ],
  "information_gaps": [
    "..."
  ]
}}

Evidence records:
{json.dumps(llm_payload, ensure_ascii=False, indent=2)}
""".strip()


prompt = build_llm_prompt(ctx, llm_payload)


# ----------------------------
# 4-5. Call OpenAI API (robust JSON parsing)
# ----------------------------
def call_llm_json(prompt: str, model: str = None) -> dict:
    """
    Call OpenAI and parse JSON robustly.
    """
    if model is None:
        model = os.getenv("OPENAI_MODEL", "gpt-4o-mini")

    # If your model supports it, prefer JSON response format.
    # If not supported, fallback to regex extraction.
    try:
        resp = client.responses.create(
            model=model,
            input=prompt,
            response_format={"type": "json_object"},
        )
        raw = (resp.output_text or "").strip()
        return json.loads(raw)
    except Exception:
        # Fallback: extract JSON block defensively
        resp = client.responses.create(model=model, input=prompt)
        raw_text = (resp.output_text or "").strip()

        import re
        match = re.search(r"\{.*\}", raw_text, flags=re.DOTALL)
        if not match:
            raise ValueError("LLM output does not contain valid JSON.")
        return json.loads(match.group(0))


llm_output = call_llm_json(prompt)


# ----------------------------
# 4-6. Inspect output
# ----------------------------
display(Markdown("#### LLM output preview"))
display(json.dumps(llm_output, indent=2, ensure_ascii=False))


# ----------------------------
# 4-7. Accessor
# ----------------------------
def get_llm_output() -> dict:
    return llm_output


### 4. LLM Synthesis

Using **7** fetched pages (snippet + extracted text).

#### LLM output preview

'{\n  "profile_summary": {\n    "current_role": "CEO",\n    "background_overview": "ファム・クアン・クオン氏は、シンガポール南洋理工大学（NTU）で神経科学とロボット工学の博士号を取得し、2018年にEureka Roboticsを共同創業しました。"\n  },\n  "career_highlights": [\n    {\n      "point": "Eureka RoboticsはシリーズAラウンドで1,050万ドルの資金調達を実施しました。",\n      "sources": [\n        "https://prtimes.jp/main/html/rd/p/000000001.000153512.html",\n        "https://robotstart.info/article/2024/12/13/365453.html"\n      ]\n    },\n    {\n      "point": "Eureka Roboticsは、主に製造業向けに高精度なロボット制御システムを提供しています。",\n      "sources": [\n        "https://ut-ec.co.jp/our_companies/eureka-robotics/"\n      ]\n    }\n  ],\n  "public_themes": [\n    {\n      "theme": "製造業におけるロボックスの自動化",\n      "evidence": [\n        "https://robotstart.info/article/2024/12/13/365453.html"\n      ]\n    }\n  ],\n  "hypotheses": [\n    {\n      "hypothesis": "ファム・クアン・クオン氏は、将来的にEureka Roboticsをさらに成長させるために、技術革新に注力すると考えられます。",\n      "rationale": "資金調達により新技術の開発と市場参入を加速するため。",\n      "sources": [\n        "http

In [8]:
# ============================================================
# 5. Output
# ============================================================
# Purpose:
# Render the structured LLM output into human-readable formats
# suitable for real-world usage (meeting brief, quick summary).
#
# Output language follows the user's selection in Section 1.
#
# This section does NOT add new information.
# It only reformats and presents what the LLM has already produced.
#
# Input:
# - llm_output from Section 4
# - InputContext (language_preference)
#
# Output:
# - Meeting-ready brief (Markdown)
# - Optional short summary for quick sharing
# ============================================================

def translate_llm_output(llm_output: dict, target_lang: str) -> dict:
    """
    Translate all human-readable fields in llm_output into the target language.
    JSON structure and keys must be preserved.
    """
    if target_lang == "en":
        return llm_output  # no-op

    prompt = f"""
Translate the following JSON content into Japanese.

Rules:
- Preserve the JSON structure and keys exactly.
- Translate only the string values meant for human reading.
- Do NOT add or remove fields.
- Keep URLs unchanged.
- Use natural business Japanese suitable for meeting preparation.

JSON:
{json.dumps(llm_output, ensure_ascii=False, indent=2)}
"""

    response = client.responses.create(
        model=os.getenv("OPENAI_MODEL", "gpt-4o-mini"),
        input=prompt,
    )

    text = response.output_text.strip()
    import re
    m = re.search(r"\{.*\}", text, re.DOTALL)
    if not m:
        raise ValueError("Translation failed: JSON not found")

    return json.loads(m.group(0))
# ----------------------------
# 5-1. Load inputs
# ----------------------------
ctx = get_input_context()
llm_output_raw = get_llm_output()
llm_output = translate_llm_output(llm_output_raw, ctx.language_preference)

lang = ctx.language_preference  # "en" or "ja"

display(Markdown("### 5. Output"))
display(Markdown(f"Rendering output in **{lang.upper()}**"))




# ----------------------------
# 5-2. Language helpers
# ----------------------------
LABELS = {
    "en": {
        "profile": "Profile summary",
        "career": "Career highlights",
        "themes": "Public themes",
        "hypotheses": "Hypotheses (to validate)",
        "talking_points": "Suggested talking points",
        "gaps": "Information gaps",
        "sources": "Sources",
        "confidence": "Confidence",
    },
    "ja": {
        "profile": "プロフィール概要",
        "career": "主な経歴・ハイライト",
        "themes": "対外的に語っているテーマ",
        "hypotheses": "仮説（要検証）",
        "talking_points": "想定トーキングポイント",
        "gaps": "未確認・追加で聞くべき点",
        "sources": "参照URL",
        "confidence": "確度",
    },
}

L = LABELS[lang]


# ----------------------------
# 5-3. Render meeting brief
# ----------------------------
def render_meeting_brief(llm_output: dict, lang: str) -> str:
    L = LABELS[lang]

    lines = []

    # --- Header ---
    lines.append(f"# Initial Meeting Brief")
    lines.append(f"**Name:** {ctx.person_name}")
    if ctx.company_name:
        lines.append(f"**Company:** {ctx.company_name}")
    lines.append("")

    # --- Profile summary ---
    ps = llm_output.get("profile_summary", {})
    lines.append(f"## {L['profile']}")
    lines.append(f"- **Current role:** {ps.get('current_role', 'not found')}")
    lines.append(f"- {ps.get('background_overview', '')}")
    lines.append("")

    # --- Career highlights ---
    lines.append(f"## {L['career']}")
    for ch in llm_output.get("career_highlights", []):
        lines.append(f"- {ch['point']}")
        for u in ch.get("sources", []):
            lines.append(f"  - {u}")
    lines.append("")

    # --- Public themes ---
    lines.append(f"## {L['themes']}")
    for th in llm_output.get("public_themes", []):
        lines.append(f"- {th['theme']}")
        for u in th.get("evidence", []):
            lines.append(f"  - {u}")
    lines.append("")

    # --- Hypotheses ---
    lines.append(f"## {L['hypotheses']}")
    for h in llm_output.get("hypotheses", []):
        lines.append(f"- **Hypothesis:** {h['hypothesis']}")
        lines.append(f"  - Rationale: {h['rationale']}")
        lines.append(f"  - {L['confidence']}: {h.get('confidence', '')}")
        for u in h.get("sources", []):
            lines.append(f"  - {u}")
    lines.append("")

    # --- Talking points ---
    lines.append(f"## {L['talking_points']}")
    for tp in llm_output.get("talking_points", []):
        lines.append(f"- {tp['question']}")
        lines.append(f"  - Intent: {tp['intent']}")
        if tp.get("related_hypothesis"):
            lines.append(f"  - Related hypothesis: {tp['related_hypothesis']}")
    lines.append("")

    # --- Information gaps ---
    lines.append(f"## {L['gaps']}")
    for g in llm_output.get("information_gaps", []):
        lines.append(f"- {g}")

    return "\n".join(lines)


meeting_brief_md = render_meeting_brief(llm_output, lang)

display(Markdown(meeting_brief_md))


# ----------------------------
# 5-4. Render short summary (optional)
# ----------------------------
def render_short_summary(llm_output: dict, lang: str) -> str:
    if lang == "ja":
        return (
            f"【事前サマリー】\n"
            f"- {llm_output.get('profile_summary', {}).get('current_role', '役割不明')}\n"
            f"- 主な論点: "
            + ", ".join([t["theme"] for t in llm_output.get("public_themes", [])[:3]])
        )
    else:
        return (
            f"[Quick summary]\n"
            f"- Role: {llm_output.get('profile_summary', {}).get('current_role', 'not found')}\n"
            f"- Key themes: "
            + ", ".join([t["theme"] for t in llm_output.get("public_themes", [])[:3]])
        )


short_summary = render_short_summary(llm_output, lang)

display(Markdown("### Quick summary"))
display(Markdown(f"```\n{short_summary}\n```"))


# ----------------------------
# 5-5. Accessors / exports
# ----------------------------
def get_meeting_brief_markdown() -> str:
    """
    Returns the meeting brief as Markdown text.
    """
    return meeting_brief_md


def get_short_summary() -> str:
    """
    Returns a short text summary for quick sharing.
    """
    return short_summary


### 5. Output

Rendering output in **JA**

# Initial Meeting Brief
**Name:** Pham Quang Cuong
**Company:** Eureka Robotics

## プロフィール概要
- **Current role:** CEO
- ファム・クアン・クオン氏は、シンガポール南洋理工大学（NTU）で神経科学とロボット工学の博士号を取得し、2018年にEureka Roboticsを共同創業しました。

## 主な経歴・ハイライト
- Eureka RoboticsはシリーズAラウンドで1,050万ドルの資金調達を実施しました。
  - https://prtimes.jp/main/html/rd/p/000000001.000153512.html
  - https://robotstart.info/article/2024/12/13/365453.html
- Eureka Roboticsは、主に製造業向けに高精度なロボット制御システムを提供しています。
  - https://ut-ec.co.jp/our_companies/eureka-robotics/

## 対外的に語っているテーマ
- 製造業におけるロボットの自動化
  - https://robotstart.info/article/2024/12/13/365453.html

## 仮説（要検証）
- **Hypothesis:** ファム・クアン・クオン氏は、将来的にEureka Roboticsをさらに成長させるために、技術革新に注力すると考えられます。
  - Rationale: 資金調達により新技術の開発と市場参入を加速するため。
  - 確度: 0.8
  - https://prtimes.jp/main/html/rd/p/000000001.000153512.html

## 想定トーキングポイント
- 今後のEureka Roboticsの技術革新の方向性は？
  - Intent: Eureka Roboticsの将来性に関する洞察を得るため。
  - Related hypothesis: ファム・クアン・クオン氏は、将来的にEureka Roboticsをさらに成長させるために、技術革新に注力すると考えられます。

## 未確認・追加で聞くべき点
- ファム・クアン・クオン氏の具体的な経歴やその他の実績についての詳細情報が不足している。

### Quick summary

```
【事前サマリー】
- CEO
- 主な論点: 製造業におけるロボットの自動化
```